In [14]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import tensorflow as tf

In [15]:
models = tf.keras.models  # like 'from tensorflow.keras import models' (PyCharm import issue workaround)
layers = tf.keras.layers  # like 'from tensorflow.keras import layers' (PyCharm import issue workaround)
optimizers = tf.keras.optimizers  # like 'from tensorflow.keras import optimizers' (PyCharm import issue workaround)

In [16]:
def load_mnist_data():
    """
    Loads the MNIST Data Set and reshapes it for further model training

    :return:
        train_images        numpy array of shape (60000, 28, 28, 1)
        train_labels        numpy array of shape (60000, )
        test_images         numpy array of shape (10000, 28, 28, 1)
        test_labels         numpy array of shape (10000, )
    """

    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

    train_images = train_images.reshape((60000, 28, 28, 1))
    test_images = test_images.reshape((10000, 28, 28, 1))

    # Normalize pixel values to be between 0 and 1
    train_images, test_images = train_images / 255.0, test_images / 255.0

    return train_images, train_labels, test_images, test_labels

In [17]:
def split_data_into_clients(clients, split, train_data, train_labels):
    """
    Utility function to split train data and labels into a specified number of clients, in accordance with a specified
    type of split.

    :param clients:                     int, number of clients the data needs to be split into
    :param split:                       string, type of split that should be performed
    :param train_data:                  numpy array, train data
    :param train_labels:                numpy array, train_labels
    :return:
        train_data                      list of numpy arrays, train_data, split into clients
        train_labels                    list of numpy arrays, train_labels, split into clients
    """

    assert len(train_data) == len(train_labels)

    # Split data
    if split.lower() == 'random':
        train_data, train_labels = fed_CNN.split_data_into_clients(clients, train_data, train_labels)
    elif split.lower() == 'overlap':
        train_data, train_labels = Data_Loader.sort_data(train_data, train_labels)
        train_data, train_labels = fed_CNN.split_data_into_clients(clients, train_data, train_labels)
        for idx in range(len(train_data)):
            train_data[idx], train_labels[idx] = Data_Loader.unison_shuffled_copies(train_data[idx], train_labels[idx])
    elif split.lower() == 'no_overlap':
        split_data, split_labels = Data_Loader.split_by_label(train_data, train_labels)
        train_data, train_labels = Data_Loader.allocate_data(clients,
                                                             split_data,
                                                             split_labels,
                                                             categories_per_client=2,
                                                             data_points_per_category=int(
                                                                 len(train_data) / (clients * 2)))
    else:
        raise ValueError(
            "Invalid value for 'Split'. Value can be 'random', 'overlap', 'no_overlap', value was: {}".format(split))
    return train_data, train_labels


In [18]:
def build_cnn(input_shape):
    """
    Compile and return a simple CNN model for image recognition.

    Configuration:
    Layer 1: Convolution Layer | Filters: 32 | Kernel Size: 3x3 | Activation: Relu
    Layer 2: Max Pooling Layer | Filter: 2x2
    Layer 3: Dense Layer       | Neurons: 32 | Activation: Relu
    Layer 4: Dense Layer       | Neurons: 10 | Activation: Softmax

    Optimizer:      Adam
    Loss function:  Sparse Categorical Cross Entropy
    Loss metric:    Accuracy


    :param input_shape:     image input shape (tuple), e.g. (28, 28, 1)

    :return:
        model               compiled tensorflow model
    """

    # Set up model type
    model = models.Sequential()

    # Add layers, inspired by https://www.tensorflow.org/beta/tutorials/images/intro_to_cnns
    model.add(layers.Conv2D(32, (5, 5), input_shape=input_shape, kernel_initializer='random_normal'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (5, 5), input_shape=input_shape, kernel_initializer='random_normal'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu', kernel_initializer='random_normal'))
    model.add(layers.Dense(10, activation='softmax', kernel_initializer='random_normal'))

    return model

In [19]:
# Build model
model_1 = build_cnn(input_shape=(28, 28, 1))
model_2 = build_cnn(input_shape=(28, 28, 1))
model_3 = build_cnn(input_shape=(28, 28, 1))
model_4 = build_cnn(input_shape=(28, 28, 1))
# Compile the model
model_1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_4.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
clients = 10
split = 'random'
train_data, train_labels, test_data, test_labels = load_mnist_data()
train_data, train_labels = split_data_into_clients(clients, split, train_data, train_labels)

NameError: name 'fed_CNN' is not defined